In [ ]:
import pandas as pd
import os
import glob
from timeit import default_timer as timer

In [5]:
pd.set_option('display.max_rows', None)

# 1. Load data

## 1.1 EPC

In [ ]:
# read data from all certificates
dir = 'data/01_raw/EPC_all'

las = os.listdir(dir)

dfs = []
chunk_size=10000

print('Loading files...')
start_time = timer()

for i in las:
    file = '{}\\{}\\certificates.csv'.format(dir,i)
    for chunk in pd.read_csv(file,chunksize=chunk_size):
        dfs.append(chunk)

epcs = pd.concat(dfs, ignore_index=True)

end_time = timer()
print("All files concatenated(Time: {:.2f} seconds)".format(end_time - start_time))

## 1.2 NSPL

In [7]:
# 2011 Census
dir = 'data/01_raw/NSPL'

geos = os.listdir(dir)

dfs = []

# load files
for geo in geos:
    path = dir + '\\' + geo

    for chunk in pd.read_csv(path, chunksize=10000):
        dfs.append(chunk)

geo_lookup = pd.concat(dfs)

In [9]:
# 2021 Census
dir = 'data/01_raw/NSPL21'

geos = os.listdir(dir)

dfs = []

# load files
for geo in geos:
    path = dir + '\\' + geo

    for chunk in pd.read_csv(path, chunksize=10000):
        dfs.append(chunk)

geo_lookup21 = pd.concat(dfs)

In [10]:
# LA code name
la = pd.read_csv('data/01_raw/LA_UA.csv')

## 1.3 Zoopla

In [ ]:
dir = 'data/01_raw/Zoopla'

years = os.listdir(dir)

dfs = []
chunk_size = 10000

# load files
print('Loading files...')
start_time = timer()

for year in years:
    path = dir + '\\' + year
    file = glob.glob(path + '/*.csv')

    print(file)

    for filename in file:
        for chunk in pd.read_csv(filename, chunksize=chunk_size):
            dfs.append(chunk)

listings = pd.concat(dfs)

end_time = timer()
print("All files concatenated(Time: {:.2f} seconds)".format(end_time - start_time))

# 2. Data preparation

## 2.1 EPC

### 2.1.1 Geography selection

In [13]:
# Preprocess postcode
geo_lookup['pcd'] = geo_lookup['pcd'].str.replace(" ", "")
geo_lookup21['pcd'] = geo_lookup21['pcd'].str.replace(" ", "")
epcs['POSTCODE'] = epcs['POSTCODE'].str.replace(" ", "")

In [14]:
# Link EPC postcode to NSPL21
epcs_geo = epcs.join(geo_lookup21.set_index('pcd')[['laua','lsoa21','oa21']], on='POSTCODE', how='left')

In [15]:
# Link LA code to name
epcs_geo_la = epcs_geo.join(la.set_index('LAD21CD')[['LAD21NM']], on='laua', how='left')

In [16]:
# Select GM LAs
epcs_geo_GM = epcs_geo_la[(epcs_geo_la['LAD21NM']=='Manchester') | (epcs_geo_la['LAD21NM']=='Stockport') | (epcs_geo_la['LAD21NM']=='Bolton') | (epcs_geo_la['LAD21NM']=='Salford') | (epcs_geo_la['LAD21NM']=='Bury') | 
                          (epcs_geo_la['LAD21NM']=='Oldham') | (epcs_geo_la['LAD21NM']=='Rochdale') | (epcs_geo_la['LAD21NM']=='Trafford') | (epcs_geo_la['LAD21NM']=='Tameside') | (epcs_geo_la['LAD21NM']=='Wigan')]

In [17]:
epcs_geo_GM['LAD21NM'].value_counts()

LAD21NM
Manchester    297107
Salford       155912
Wigan         132936
Bolton        120524
Stockport     115922
Tameside      104199
Oldham         99291
Rochdale       95156
Trafford       93123
Bury           81644
Name: count, dtype: int64

In [20]:
# Link to NSPL11 - 100% linkage success
epcs_geo_GM_11 = epcs_geo_GM.join(geo_lookup.set_index('pcd')[['lsoa11','msoa11']], on='POSTCODE', how='left')

### 2.2.2 UPRN filtering

In [21]:
epcs_uprn = epcs_geo_GM_11.dropna(subset=['UPRN'])

In [22]:
epcs_uprn.shape, epcs_uprn.shape[0]/epcs_geo_GM_11.shape[0]

((1278020, 98), 0.9862680909451511)

### 2.2.3 Deduplication

#### EPC with same inspection date - keep last

In [23]:
epcs_uprn['INSPECTION_DATE'] = pd.to_datetime(epcs_uprn['INSPECTION_DATE'])

epcs_uprn['LODGEMENT_DATE'] = pd.to_datetime(epcs_uprn['LODGEMENT_DATE'])

C:\Users\2702471O\AppData\Local\Temp\ipykernel_19768\684049633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epcs_uprn['INSPECTION_DATE'] = pd.to_datetime(epcs_uprn['INSPECTION_DATE'])
C:\Users\2702471O\AppData\Local\Temp\ipykernel_19768\684049633.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epcs_uprn['LODGEMENT_DATE'] = pd.to_datetime(epcs_uprn['LODGEMENT_DATE'])


In [24]:
# Sort by date to ensure "last" record is defined in chronological order
epcs_uprn = epcs_uprn.sort_values(by=['INSPECTION_DATE','LODGEMENT_DATE'])

In [25]:
# Deduplicate UPRN-same date
epcs_uprn.shape

(1278020, 98)

In [31]:
# drop duplication on UPRN-inspection date - keep last
epcs_dedu_insp = epcs_uprn.drop_duplicates(subset=['UPRN','INSPECTION_DATE'], keep='last')

In [32]:
epcs_dedu_insp.shape

(1256158, 98)

#### EPC with lodgement date within one week - keep last

In [33]:
df_dedu_lodg = epcs_dedu_insp.sort_values(by=['UPRN', 'LODGEMENT_DATE']).copy()

In [34]:
# Mark rows within 7 days of the next entry in the group as duplicates
# 1. Calculate the difference in days between each row and the previous row within each group
df_dedu_lodg['date_diff'] = df_dedu_lodg.groupby('UPRN')['LODGEMENT_DATE'].diff().dt.days.fillna(float('inf'))

# 2. Assign a "keep" flag based on whether the difference is 7 days or more (this will keep the last occurrence)
df_dedu_lodg['keep'] = (df_dedu_lodg['date_diff'] >= 7) | (df_dedu_lodg['date_diff'] == float('inf'))

# 3. Only keep rows marked as `True` for "keep"
epcs_dedu = df_dedu_lodg[df_dedu_lodg['keep']].drop(columns=['date_diff', 'keep'])

In [35]:
epcs_dedu.shape, epcs_dedu.shape[0]/epcs_uprn.shape[0], epcs_dedu.shape[0]/epcs_dedu_insp.shape[0]

((1249288, 98), 0.9775183486956386, 0.9945309427635696)

#### Property type

In [40]:
grouped_df = epcs_dedu.groupby('UPRN').agg({'PROPERTY_TYPE': list})
grouped_df['PROPERTY_TYPE'].value_counts()

PROPERTY_TYPE
[House]                                                                   409990
[House, House]                                                            144007
[Flat]                                                                    142022
[Flat, Flat]                                                               67423
[Bungalow]                                                                 36418
[House, House, House]                                                      29852
[Flat, Flat, Flat]                                                         12353
[Bungalow, Bungalow]                                                        9951
[House, House, House, House]                                                6101
[Maisonette]                                                                5789
[House, Flat]                                                               3405
[Flat, Flat, Flat, Flat]                                                    2135
[Maisonette, F

In [41]:
# Percentage of property type changing - 1.46% of UPRNs have property type change
# Step 1: Group by 'House_ID' and find unique property types
unique_property_types = epcs_dedu.groupby('UPRN')['PROPERTY_TYPE'].nunique()

# Step 2: Filter for properties with changes (more than 1 unique property type)
changed_properties = unique_property_types[unique_property_types > 1].count()

# Step 3: Calculate the total number of properties
total_properties = unique_property_types.count()

# Step 4: Calculate the percentage of property type changes
change_percentage = (changed_properties / total_properties) * 100

change_percentage

1.4580866468413585

In [42]:
epcs_dedu['PROPERTY_TYPE'].value_counts()

PROPERTY_TYPE
House         829717
Flat          337669
Bungalow       67376
Maisonette     14440
Park home         86
Name: count, dtype: int64

In [43]:
# Percentage change if aggregate to house/flat
epcs_dedu['PROPERTY_TYPE_AGG'] = epcs_dedu['PROPERTY_TYPE'].apply(lambda x: 'House' if x in ['House','Bungalow'] else ('Flat' if x in ['Flat','Maisonette'] else x))

In [44]:
# Step 1: Group by 'House_ID' and find unique property types
unique_property_types = epcs_dedu.groupby('UPRN')['PROPERTY_TYPE_AGG'].nunique()

# Step 2: Filter for properties with changes (more than 1 unique property type)
changed_properties = unique_property_types[unique_property_types > 1].count()

# Step 3: Calculate the total number of properties
total_properties = unique_property_types.count()

# Step 4: Calculate the percentage of property type changes
change_percentage = (changed_properties / total_properties) * 100

change_percentage

0.647071850171287

In [45]:
changed_properties

5727

#### Bulit form

In [46]:
# Percentage of property type changing
# Step 1: Group by 'House_ID' and find unique property types
unique_property_types = epcs_dedu.groupby('UPRN')['BUILT_FORM'].nunique()

# Step 2: Filter for properties with changes (more than 1 unique property type)
changed_properties = unique_property_types[unique_property_types > 1].count()

# Step 3: Calculate the total number of properties
total_properties = unique_property_types.count()

# Step 4: Calculate the percentage of property type changes
change_percentage = (changed_properties / total_properties) * 100

change_percentage

8.825689441667494

In [47]:
grouped_df = epcs_dedu.groupby('UPRN').agg({'BUILT_FORM': list})
grouped_df['BUILT_FORM'].value_counts()

BUILT_FORM
[Semi-Detached]                                                                                                                                                     215956
[Mid-Terrace]                                                                                                                                                       164416
[Detached]                                                                                                                                                          101823
[End-Terrace]                                                                                                                                                        86968
[Mid-Terrace, Mid-Terrace]                                                                                                                                           68447
[Semi-Detached, Semi-Detached]                                                                                                        

In [48]:
# Percentage of UPRNs have null built form at all
11957/grouped_df.shape[0]

0.013509757486464255

#### EPC rating

In [49]:
grouped_df = epcs_dedu.groupby('UPRN').agg({'CURRENT_ENERGY_RATING': list})
grouped_df['CURRENT_ENERGY_RATING'].value_counts()

CURRENT_ENERGY_RATING
[D]                                  230584
[C]                                  193255
[B]                                   84540
[E]                                   70967
[C, C]                                49743
[D, D]                                46447
[D, C]                                33787
[E, D]                                20874
[C, D]                                16038
[F]                                   11507
[B, C]                                 9987
[E, E]                                 9143
[D, E]                                 8317
[E, C]                                 8217
[D, D, D]                              5489
[C, C, C]                              5392
[F, D]                                 4786
[B, B]                                 4718
[C, B]                                 4592
[D, D, C]                              4113
[F, E]                                 2782
[G]                                    2681
[D, C, C] 

#### Habitable rooms

In [50]:
epcs_dedu['NUMBER_HABITABLE_ROOMS'] = epcs_dedu['NUMBER_HABITABLE_ROOMS'].astype('Int64').apply(str)

In [51]:
epcs_dedu['NUMBER_HABITABLE_ROOMS'].value_counts()

NUMBER_HABITABLE_ROOMS
4.0       311934
5.0       275055
3.0       223260
2.0       147207
nan       114342
6.0        89318
7.0        42439
8.0        18312
1.0        13803
9.0         7468
10.0        3264
11.0        1329
12.0         678
13.0         305
14.0         154
15.0         103
16.0          75
17.0          47
18.0          32
20.0          20
19.0          18
40.0          12
33.0          10
55.0           8
22.0           8
0.0            6
50.0           6
23.0           5
30.0           5
45.0           5
21.0           4
35.0           4
25.0           4
24.0           4
43.0           3
77.0           3
27.0           3
42.0           3
75.0           2
44.0           2
41.0           2
63.0           2
38.0           2
26.0           2
31.0           1
74.0           1
32.0           1
3424.0         1
56.0           1
37.0           1
60.0           1
34.0           1
76.0           1
97.0           1
51.0           1
84.0           1
72.0           1
54.0    

In [52]:
grouped_df = epcs_dedu.groupby('UPRN').agg({'NUMBER_HABITABLE_ROOMS': list})
grouped_df['NUMBER_HABITABLE_ROOMS'].value_counts()

NUMBER_HABITABLE_ROOMS
[4.0]                                                      136512
[5.0]                                                      136352
[nan]                                                       93829
[3.0]                                                       83671
[2.0]                                                       53340
[6.0]                                                       46636
[4.0, 4.0]                                                  37761
[3.0, 3.0]                                                  34787
[5.0, 5.0]                                                  29948
[2.0, 2.0]                                                  27196
[7.0]                                                       23078
[4.0, 5.0]                                                  12391
[5.0, 4.0]                                                  10392
[8.0]                                                        9775
[3.0, 4.0]                                           

In [54]:
# Percentage of errors of all UPRNs
# Step 1: Group by 'House_ID' and find unique property types
unique_property_types = epcs.groupby('UPRN')['NUMBER_HABITABLE_ROOMS'].nunique()

# Step 2: Filter for properties with changes (more than 1 unique property type)
changed_properties = unique_property_types[unique_property_types > 1].count()

# Step 3: Calculate the total number of properties
total_properties = unique_property_types.count()

# Step 4: Calculate the percentage of property type changes
change_percentage = (changed_properties / total_properties) * 100

change_percentage

11.25903267266381

### 2.2.4 Save data

In [55]:
epcs_dedu.to_csv('data/02_intermediate/epc.csv.gz', index=False, compression='gzip',chunksize=10000)

## 2.2 Zoopla

### 2.2.1 Geography/time check

In [57]:
listings.shape

(1977570, 42)

In [58]:
# geography check
listings['nspl_local_authority'].value_counts()

nspl_local_authority
Manchester    521257
Salford       393021
Wigan         200850
Bolton        195556
Bury          136142
Stockport     128267
Trafford      126044
Tameside      119328
Rochdale       84538
Oldham         72567
Name: count, dtype: int64

In [59]:
# time check
listings['date'] = pd.to_datetime(listings['date'])

listings['date'].min(), listings['date'].max()

(Timestamp('2017-09-01 00:00:00'), Timestamp('2024-05-31 00:00:00'))

### 2.2.2 Deduplication

In [60]:
# Deduplicate on listing ID - keep last
listings['date'].is_monotonic_increasing

True

In [61]:
listings_dedu = listings.drop_duplicates(subset=['listing_id'], keep='last')

In [62]:
# rows and percentage
listings_dedu.shape, listings_dedu.shape[0]/listings.shape[0]

((594522, 42), 0.3006325945478542)

### 2.2.3 Sample selection 1 - residential

In [63]:
listings_dedu['category'].value_counts()

category
residential    577675
commercial      16838
Name: count, dtype: int64

In [64]:
listings_dedu_res = listings_dedu[listings_dedu['category']=='residential'].copy()

In [65]:
# rows and percentage
listings_dedu_res.shape, listings_dedu_res.shape[0]/listings_dedu.shape[0]

((577675, 42), 0.9716629493946397)

### 2.2.4 Sample selection 2 - sales

In [66]:
listings_dedu_res['transaction_type'].value_counts()

transaction_type
sale    356705
rent    220922
Name: count, dtype: int64

In [67]:
listings_dedu_res_sales = listings_dedu_res[listings_dedu_res['transaction_type']=='sale']

In [68]:
# rows and percentage
listings_dedu_res_sales.shape, listings_dedu_res_sales.shape[0]/listings_dedu_res.shape[0]

((356705, 42), 0.6174838793439218)

### 2.2.5 Sample selction 3 - auction/buyer incentives

In [69]:
listings_s2 = listings_dedu_res_sales.copy()

In [70]:
listings_s2['detailed_description'] = listings_s2['detailed_description'].astype(str)

In [71]:
# 1. add variable on auction
auction_keywords = ['public auction','auction']

def is_auction(text):
    # Convert the text to lowercase for case-insensitive matching
    text = text.lower()
    # Check if any of the keywords are in the text
    for keyword in auction_keywords:
        if keyword in text:
            return True
    return False

In [72]:
listings_s2['is_auction'] = listings_s2['detailed_description'].apply(is_auction)

In [73]:
# 2. add variable on share ownership
shareo_keywords = ['share ownership','shared ownership']

def is_shareo(text):
    # Convert the text to lowercase for case-insensitive matching
    text = text.lower()
    # Check if any of the keywords are in the text
    for keyword in shareo_keywords:
        if keyword in text:
            return True
    return False

In [74]:
listings_s2['is_shareo'] = listings_s2['detailed_description'].apply(is_shareo)

In [75]:
listings_s2_nauc = listings_s2[listings_s2['is_auction']==False]

In [76]:
listings_s2_nauc.shape,listings_s2[listings_s2['is_auction']==False].shape

((348576, 44), (348576, 44))

In [77]:
listings_s2_nauc_nshar = listings_s2_nauc[listings_s2_nauc['is_shareo']==False]

In [78]:
listings_s2_nauc_nshar.shape, listings_s2_nauc_nshar.shape[0]/listings_s2.shape[0]

((346915, 44), 0.9725543516351046)

### 2.2.6 UPRN filtering

In [80]:
listings_s3 = listings_s2_nauc_nshar.copy()

In [ ]:
# drop null UPRN
listings_dedu_uprn = listings_s3.dropna(subset=['uprn'])

In [83]:
listings_dedu_uprn.shape, listings_dedu_uprn.shape[0]/listings_s3.shape[0]

((235425, 44), 0.6786244469106265)

In [84]:
# listings with same UPRN within one week
listings_uprn_ldate = listings_dedu_uprn.sort_values(by=['uprn', 'date']).copy()

In [85]:
# Mark rows within 7 days of the next entry in the group as duplicates
# 1. Calculate the difference in days between each row and the previous row within each group
listings_uprn_ldate['date_diff'] = listings_uprn_ldate.groupby('uprn')['date'].diff().dt.days.fillna(float('inf'))

# 2. Assign a "keep" flag based on whether the difference is 7 days or more (this will keep the last occurrence)
listings_uprn_ldate['keep'] = (listings_uprn_ldate['date_diff'] >= 7) | (listings_uprn_ldate['date_diff'] == float('inf'))

# 3. Only keep rows marked as `True` for "keep"
listings_dedu_date = listings_uprn_ldate[listings_uprn_ldate['keep']].drop(columns=['date_diff', 'keep'])

In [86]:
listings_dedu_date.shape, listings_dedu_date.shape[0]/listings_uprn_ldate.shape[0]

((231107, 44), 0.9816587023468196)

### 2.2.7 Save data

In [88]:
listings_dedu_date.to_csv('data/02_intermediate/listings_prep.csv')